## Sun B1 dynamics: nastier than expected...
When we have 3BP dynamics in E-M rotating frame, we naturally forget about $\omega$ as we normalize t* so that $\omega$ = 1 (i.e., $t_1^*$ = 2$\pi$ / (1 siderial period = 27.5days). 
This means omega should change if we want to discuss the dynamics in Sun-B1 frame ($t_2^*$ = 2$\pi$ / 365 days), while we want to stick with $t_1^*$. Therefore, the angular velocity term in the centrifugal and Coriolis force emerges: $\omega_b = 2\pi$ / (365 days / 27.5 days).

Angular veclocity $\omega_b$ is $\omega_b = 2\pi / t_{synodic}$.




### normalization
$t^* = t_{syderial} = 27.5 * 86400$ [s], 

$l^*$ = Earth-Moon distance,

$m^* = m_E + m_M$.

In [1]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyBase not found in current path:
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
│    
└ @ Plots C:\Users\yujit\.julia\packages\Plots\nuwp4\src\backends.jl:545


Plots.PlotlyBackend()

In [13]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")
param3b = SailorMoon.dynamics_parameters()


┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 0.9251999994040079, 0.07480000059599223, 0.01709689063726318, 7.601281331451572)

In [14]:
"""
Right-hand side expression for state-vector in BCR4BP
# Arguments
    - `du`: cache array of duative of state-vector, mutated
    - `u`: state-vector
    - `p`: parameters, where p = [μ2, μS, as, θ0, ωM, τ, γ, β, mdot, tmax]
        m* : mE + mL (6.0455 * 10^22)
        μ2 : mL / m* (0.012150585609624)
        μS : mS / m* (0.00000303951)
        as : (sun-B1 distance) / l* (388.709677419)
            l* : Earth-moon distance (384,400 km)
        ωM : Earth-Moon line's angular velocity around E-M barycenter
        τ  : thrust magnitude (0~1)
        γ  : thrust angle 1
        β  : thrust angle 2
        mdot : mass-flow rate
        tmax : max thrust
    - `t`: time
"""
function rhs_bcr4bp_sb1frame!(du,u,p,t)
    # unpack state
    x, y, z = u[1], u[2], u[3]
    vx, vy, vz = u[4], u[5], u[6]
    μ2, μS, as, θ0, ωM, ωb = p[1], p[2], p[3], p[4], p[5], p[6]
    τ, γ, β, mdot, tmax = p[7], p[8], p[9], p[10], p[11]
    
    θ = θ0 + ωM * t  # moon angle

    # create Thrust term
#     T = dv_inertial_angles([τ,γ,β])
    T = [0.0, 0.0, 0.0]
    Tx, Ty, Tz = T * tmax / u[7]  # mdot


    # derivatives of positions
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]

    # earth and moon location
    xe = - μ2 *cos(θ) + as * μS/(μS+1) 
    ye = - μ2 *sin(θ)
    ze = 0

    xm = (1-μ2) *cos(θ) + as * μS/(μS+1) 
    ym = (1-μ2) *sin(θ)
    zm = 0
#     println(xe, " ", ye, " ", ze)

    # compute distances
    r30 = sqrt((-as/(μS+1) - x)^2 + y^2 + z^2)      # SC-Sun
    r31 = sqrt((xe - x)^2 + (ye - y)^2 + (ze-z)^2)  # SC-earth
    r32 = sqrt((xm - x)^2 + (ym - y)^2 + (zm-z)^2)  # SC-Moon
#     println(r30, " ", r31, " ", r32)

    Fx = -(μS)*(x+as/(μS+1))/r30^3 - (1-μ2)*(x-xe)/r31^3 - μ2*(x-xm)/r32^3 + Tx
    Fy = -(μS)* y /r30^3           - (1-μ2)*(y-ye)/r31^3 - μ2*(y-ym)/r32^3 + Ty
    Fz = -(μS)* z /r30^3           - (1-μ2)*(z-ze)/r31^3 - μ2*(z-zm)/r32^3 + Tz
#     println(-μS*(x-1/(μS+1))/r30^3 , " ", - (1-μ2)*(x-xe)/r31^3, " ",  - μ2*(x-xm)/r32^3)    
    
    ωb = 0.07480000059599223
    du[4] =  2*ωb*vy + ωb^2*x + Fx
    du[5] = -2*ωb*vx + ωb^2*y + Fy
    du[6] =                   + Fz

    du[7] = -mdot * τ
end

rhs_bcr4bp_sb1frame!

In [15]:
"""
Assumption: B1 rotates around Sun (not B2)...
This is for making the dynamics compatible to the EM rotating frame's BCR4BP
"""
function rhs_bcr4bp_sb1frame2!(du,u,p,t)
    # unpack state
    x, y, z = u[1], u[2], u[3]
    vx, vy, vz = u[4], u[5], u[6]
    μ2, μS, as, θ0, ωM, ωb = p[1], p[2], p[3], p[4], p[5], p[6]
    τ, γ, β, mdot, tmax = p[7], p[8], p[9], p[10], p[11]

    θ = θ0 + ωM * t  # moon angle

    # create Thrust term
#     T = dv_inertial_angles([τ,γ,β])
    T = [0.0, 0.0, 0.0]
    Tx, Ty, Tz = T * tmax / u[7]  # mdot


    # derivatives of positions
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]

    # earth and moon location
    xe = - μ2 *cos(θ) + as 
    ye = - μ2 *sin(θ)
    ze = 0

    xm = (1-μ2) *cos(θ) + as
    ym = (1-μ2) *sin(θ)
    zm = 0
#     println(xe, " ", ye, " ", ze)

    # compute distances
    r30 = sqrt(x^2 + y^2 + z^2)      # SC-Sun
    r31 = sqrt((xe - x)^2 + (ye - y)^2 + (ze-z)^2)  # SC-earth
    r32 = sqrt((xm - x)^2 + (ym - y)^2 + (zm-z)^2)  # SC-Moon
#     println(r30, " ", r31, " ", r32)

    Fx = -(μS)*(x)/r30^3 - (1-μ2)*(x-xe)/r31^3 - μ2*(x-xm)/r32^3 + Tx
    Fy = -(μS)* y /r30^3           - (1-μ2)*(y-ye)/r31^3 - μ2*(y-ym)/r32^3 + Ty
    Fz = -(μS)* z /r30^3           - (1-μ2)*(z-ze)/r31^3 - μ2*(z-zm)/r32^3 + Tz
#     println(-μS*(x-1/(μS+1))/r30^3 , " ", - (1-μ2)*(x-xe)/r31^3, " ",  - μ2*(x-xm)/r32^3)    
    

    ωb = 0.07480000059599223
    du[4] =  2*ωb*vy + ωb^2*x + Fx
    du[5] = -2*ωb*vx + ωb^2*y + Fy
    du[6] =                   + Fz

    du[7] = -mdot * τ
end

rhs_bcr4bp_sb1frame2!

In [16]:
function rhs_bcr4bp_thrust!(du,u,p,t)
    # unpack arguments
    μ, μ_3, t0, a_s, ω_s, τ, θ, β, mdot, tmax = p
    # decompose state
    x, y, z, vx, vy, vz, mass = u

    # calculate radii
    r1 = sqrt( (x+μ)^2 + y^2 + z^2 )
    r2 = sqrt( (x-1+μ)^2 + y^2 + z^2 )

    # sun position
    xs = a_s*cos(ω_s*t + t0)
    ys = a_s*sin(ω_s*t + t0)
    zs = 0.0
    r3 = sqrt( (x-xs)^2 + (y-ys)^2 + (z-zs)^2 )

    # compute delta-V vector
#     dir_v = dv_lvlh2inertial(u[1:6], [τ, θ, β])
    dir_v = [0.0, 0.0, 0.0]
    ts = tmax*dir_v

    # position-state derivative
    du[1] = vx
    du[2] = vy
    du[3] = vz

    # velocity derivatives: Earth, moon, sun
    du[4] =  2*vy + x - ((1-μ)/r1^3)*(μ+x)  + (μ/r2^3)*(1-μ-x)  + ( -(μ_3/r3^3)*(x-xs) - (μ_3/a_s^3)*xs ) + ts[1]/mass
    du[5] = -2*vx + y - ((1-μ)/r1^3)*y      - (μ/r2^3)*y        + ( -(μ_3/r3^3)*(y-ys) - (μ_3/a_s^3)*ys ) + ts[2]/mass
    du[6] =           - ((1-μ)/r1^3)*z      - (μ/r2^3)*z        + ( -(μ_3/r3^3)*(z)    - (μ_3/a_s^3)*zs ) + ts[3]/mass
    
#     println(r1, " ", r2, " ", r3)
#     println(((1-μ)/r1^3)*(μ+x), " ", (μ/r2^3)*(1-μ-x), " ",  -(μ_3/r3^3)*(x-xs) )

    # mass derivative
    du[7] = -mdot*τ
end

rhs_bcr4bp_thrust! (generic function with 1 method)

In [17]:
"""
    transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)

Transform state from Earth-Moon rotating frame to Sun-B1 rotating frame.
Careful with sign of ωs!! (should be negative)
"""
function transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)
    ωm = -ωs
    θm = π - θs
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    C = [
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
         ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
         0          0         0 0       0      1
    ]
    state_conv = C * state
    state_conv = state_conv + [param3b.as, 0,0,0,0,0]
    
    return state_conv
end


transform_EMrot_to_SunB1

In [18]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [19]:
θM = 180 / 180 * pi  
ϕ0 = 0.0 # 5.27787565803085   #0.0
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
ωb = 2*pi*(t_synodic - t_sidereal)/t_synodic/t_sidereal*param3b.tstar
ωM = 2*pi / (t_synodic / param3b.tstar)
param = [param3b.mu2, param3b.mus, param3b.as, θM, ωM, ωb, 0.0, 0.0, 0.0, 0.0, 0.0]
println(param3b.oml)

omegaM = 2pi/(t_synodic/param3b.tstar)

0.9251999994040079


0.9251999994040079

In [20]:
# initial condition: Halo orbit progapation
lps = SailorMoon.lagrange_points(param3b.mu2)

## set up of initial condition (Lyapunov orbit)
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

# xf in the Earth-moon rotating frame
xf_em = SailorMoon.set_terminal_state2(ϕ0, θM, param3b, LPOArrival)

# change coordination from EM rot. frame to Sun-B1 frame
θs = π - θM
ωs = param3b.oms
xf_sb1 = vcat(transform_EMrot_to_SunB1(xf_em, θs, ωs), 1.0)
println(xf_sb1)

Halo guess Az_km: 1200.0
Unrecognized keyword arguments: 

┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s

[:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

Unrecognized keyword arguments: [:method]

┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s



Linear stability ν = 618.7618470972243
[387.6985504060269, -2.7938137447750573e-7, 1.2936425886039821e-23, 1.0294967930417525e-6, -1.203809978563241, 5.48460305147664e-32, 1.0]


In [21]:

# propagate ODE
tspan = [0.0, -110 * 86400 / param3b.tstar]
prob = ODEProblem(rhs_bcr4bp_sb1frame!, xf_sb1, tspan, param)
sol = solve(prob, Tsit5(), reltol=1e-12, abstol=1e-12);

prob2 = ODEProblem(rhs_bcr4bp_sb1frame2!, xf_sb1, tspan, param)
sol2 = solve(prob2, Tsit5(), reltol=1e-12, abstol=1e-12);

p=[param3b.mu2, param3b.mus, π-θM, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
xf_em = vcat(xf_em, 1.0)
prob3 = ODEProblem(rhs_bcr4bp_thrust!, xf_em, tspan, p)
sol3 = solve(prob3, Tsit5(), reltol=1e-12, abstol=1e-12);

for k in 1:size(sol3)[2]
    t = sol3.t[k]
    θs = param3b.oms*t
    sol3.u[k][1:6] = transform_EMrot_to_SunB1(sol3.u[k][1:6], θs, param3b.oms)
#     print(solution[1:6,k])
end


In [22]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="S-B1 rot.", linestyle=:solid)
plot!(pcart, Array(sol2)[1,:], Array(sol2)[2,:], color=:red, linewidth=1.0, label="S-B1 rot. (reduced)", linestyle=:solid)
plot!(pcart, Array(sol3)[1,:], Array(sol3)[2,:], color=:lime, linewidth=1.0, label="E-M rot.", linestyle=:solid)



moon = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth = plot_circle(param3b.mu2, param3b.as, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0)

plot!(pcart, earth[1,:], earth[2,:], c=:black, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:dimgrey, lw=1.0, label="moon")
# plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

In [23]:
println(ωb)
println(sqrt((param3b.mus + 1)/(param3b.as)^2))
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
println((2*pi*(t_synodic - t_sidereal)/t_synodic)/t_sidereal*param3b.tstar)

0.07480000059599223
1.4749693207153063
0.07480000059599223
